In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/namnn12/personal/nckh-2122-crawl-stock-data/data/skds.csv")

In [3]:
df

,title,content,date,source
0,"Xác minh thông tin nhiều trẻ em bị ép vào ""đườ...",\r\n SKĐS - Liê...,20200116,skds
1,"Quảng Bình: Một người chết, ba người bị thương...",\r\n SKĐS - Tin...,20200109,skds
2,"Tổng Bí thư, Chủ tịch nước Nguyễn Phú Trọng gử...",\r\n SKĐS - Tổn...,20200904,skds
3,Hải Trà Tân Cương- Trà Thái Nguyên được trồng ...,\r\n SKĐS - “Đá...,20200903,skds
4,Trường Cao đẳng Y Dược Tuệ Tĩnh Hà Nội: Hành t...,\r\n SKĐS - Mặc...,20200902,skds
...,...,...,...,...
1482,"Ra Bắc rồi vào Nam chống dịch, chàng trai 9X l...",\r\n SKĐS - Sau...,20210830,skds
1483,Điều tra nguyên nhân vụ sập mỏ đá ở Hà Nam khi...,\r\n SKĐS - Phò...,20210719,skds
1484,TP.HCM dự kiến mở rộng hỗ trợ đối tượn...,\r\n SKĐS - Ngà...,20210719,skds
1485,Tổ công tác Bộ Y tế chỉ ra giải pháp trong phò...,\r\n SKĐS - Việ...,20210719,skds


In [9]:
date_df = pd.read_csv("/home/namnn12/personal/nckh-2122-crawl-stock-data/date.txt", sep="\t")
date_df.sample

<bound method NDFrame.sample of     STT      From        To
0     1  20200120  20200124
1     2  20200128  20200201
2     3  20200205  20200209
3     4  20202017  20202020
4     5  20200222  20200226
..  ...       ...       ...
66   67  20211202  20211202
67   68  20211203  20211205
68   69  20211206  20211206
69   70  20211207  20211209
70   71  20211212  20211216

[71 rows x 3 columns]>

In [17]:
for date in date_df.itertuples():
    print(date)
    print(date.From)
    print(df[(df["date"] > date.From) & (df["date"] < date.To)])
    break

Pandas(Index=0, STT=1, From=20200120, To=20200124)
20200120
                                  title  \
1068  Tết năm nay thời tiết sẽ thế nào?   

                                                content      date source  
1068  \r\n                                SKĐS - Tru...  20200121   skds  


In [32]:
def filter_date(df):
    df_filter = df.copy()
    df_filter["event_id"] = 0
    date_df = pd.read_csv("/home/namnn12/personal/nckh-2122-crawl-stock-data/date.txt", sep="\t")
    for date in date_df.itertuples():
        df_filter.loc[(df_filter["date"] > date.From) & (df_filter["date"] < date.To), "event_id"]=date.STT

    return df_filter[df_filter['event_id']!=0]

In [33]:
filter_date(df)

,title,content,date,source,event_id
4,Trường Cao đẳng Y Dược Tuệ Tĩnh Hà Nội: Hành t...,\r\n SKĐS - Mặc...,20200902,skds,31
5,Lý do Hà Nội tạm dừng thí điểm trông xe bằng ứ...,\r\n SKĐS - Sở ...,20200901,skds,31
6,"9 nhóm nhiệm vụ chủ yếu, 5 giải pháp cơ bản nă...",\r\n SKĐS - Bộ ...,20200901,skds,31
34,Nơi nào của Hà Nội học sinh chưa thể đến trườn...,\r\n SKĐS - Tín...,20211204,skds,68
35,"Quỹ ""tiếp sức"" cho học sinh nghèo Hà Tĩnh: Chủ...",\r\n Với mục đí...,20211204,skds,68
...,...,...,...,...,...
1477,[Ảnh] Người ngoại tỉnh đổ về Thủ đô sau lệnh n...,\r\n SKĐS - Đôn...,20210922,skds,61
1483,Điều tra nguyên nhân vụ sập mỏ đá ở Hà Nam khi...,\r\n SKĐS - Phò...,20210719,skds,58
1484,TP.HCM dự kiến mở rộng hỗ trợ đối tượn...,\r\n SKĐS - Ngà...,20210719,skds,58
1485,Tổ công tác Bộ Y tế chỉ ra giải pháp trong phò...,\r\n SKĐS - Việ...,20210719,skds,58


In [49]:
def filter_content(df):
    covid_word_path="/home/namnn12/personal/nckh-2122-crawl-stock-data/covid_word.txt"
    with open(covid_word_path) as f:
        list_word = [line.replace("\n","") for line in f.readlines()]

    is_covid_news = list()
    df_filter = df.copy()
    for row in df_filter.itertuples():
        is_covid_news.append(any(ele in row.content.lower() for ele in list_word))

    return df_filter[is_covid_news]

In [50]:
filter_content(df)

,title,content,date,source
2,"Tổng Bí thư, Chủ tịch nước Nguyễn Phú Trọng gử...",\r\n SKĐS - Tổn...,20200904,skds
6,"9 nhóm nhiệm vụ chủ yếu, 5 giải pháp cơ bản nă...",\r\n SKĐS - Bộ ...,20200901,skds
7,Hà Nội dừng tất cả các lễ hội đầu năm,\r\n SKĐS - Tại...,20210214,skds
11,Chí Linh ngày 29 Tết,\r\n SKĐS - TP ...,20210210,skds
12,"Người Hà Nội tới Vinmart, Bệnh viện Thanh Nhàn...",\r\n SKĐS - Ngư...,20211212,skds
...,...,...,...,...
1481,"Xã, phường, thị trấn phải đảm bảo thu dung, ch...",\r\n SKĐS - Đó ...,20210831,skds
1482,"Ra Bắc rồi vào Nam chống dịch, chàng trai 9X l...",\r\n SKĐS - Sau...,20210830,skds
1484,TP.HCM dự kiến mở rộng hỗ trợ đối tượn...,\r\n SKĐS - Ngà...,20210719,skds
1485,Tổ công tác Bộ Y tế chỉ ra giải pháp trong phò...,\r\n SKĐS - Việ...,20210719,skds
